In [ ]:
# 导入基本的一些库
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from tqdm import tqdm_notebook
import datetime
import os
import random

from joblib import Parallel, delayed
%env JOBLIB_TEMP_FOLDER=/tmp

import scipy as sc
from scipy import stats
from scipy import signal
from scipy import fftpack 
from scipy import optimize
from scipy import fftpack
import pywt

from tsfresh.feature_extraction import feature_calculators
import librosa
import librosa.display as ld

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans


In [ ]:
%%time
#读取数据
PATH = "../input/LANL-Earthquake-Prediction/"
train = pd.read_pickle("../input/lanl-training-as-pickle/train.pkl")
# 清理内存
gc.collect()

In [ ]:
# DENOISING
# 信号长度
SIGNAL_LEN = 150000
# 采样率
SAMPLE_RATE = 4000
# 信号长度
SIG_LEN = 150000
#每次处理的信号个数
NUM_SEG_PER_PROC = 4000
# 使用的线程的个数
NUM_THREADS = 6

# 奈奎斯特采样频率
NY_FREQ_IDX = 75000  # 信号个数/2，如果比这个大，那就会出现信号失真的现象
# 截断频率
CUTOFF = 18000
# 最高频率
MAX_FREQ_IDX = 20000
# 频率窗大小
FREQ_STEP = 2500

# 平均绝对值偏差
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

# 高通滤波器
def high_pass_filter(x, low_cutoff=1000, SAMPLE_RATE=SAMPLE_RATE):
    
    # 计算奈奎斯特频率 0.5*采样率
    nyquist = 0.5 * SAMPLE_RATE
    # 使用奈奎斯特频率归一化的截止频率
    norm_low_cutoff = low_cutoff / nyquist
    
    #使用sos滤波器进行高通滤波，滤除噪音
    sos = signal.butter(10, Wn=[norm_low_cutoff], btype='highpass', output='sos')
    filtered_sig = signal.sosfilt(sos, x)

    return filtered_sig

# 信号去噪，这个采用别人的方案
def denoise_signal(x, wavelet='db4', level=1):
    """
    1. Adapted from waveletSmooth function found here:
    http://connor-johnson.com/2016/01/24/using-pywavelets-to-remove-high-frequency-noise/
    2. Threshold equation and using hard mode in threshold as mentioned
    in section '3.2 denoising based on optimized singular values' from paper by Tomas Vantuch:
    http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    """
    
    # Decompose to get the wavelet coefficients
    coeff = pywt.wavedec(x, wavelet, mode="per")
    
    # Calculate sigma for threshold as defined in http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    # As noted by @harshit92 MAD referred to in the paper is Mean Absolute Deviation not Median Absolute Deviation
    sigma = (1/0.6745) * maddest(coeff[-level])

    # Calculate the univeral threshold
    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    
    # Reconstruct the signal using the thresholded coefficients
    return pywt.waverec(coeff, wavelet, mode='per')

# 移动平均法,periods的大小为平均的周期，移动平均法用来反映趋势，去掉噪声带来的干扰
def moving_average(data_set, periods=3):
    weights = np.ones(periods) / periods
    return np.convolve(data_set, weights, mode='same')

# 4 阶巴特沃斯低通滤波器
def des_bw_filter_lp(cutoff=CUTOFF):  # low pass filter
    b, a = signal.butter(4, Wn=cutoff/NY_FREQ_IDX)
    return b, a

# 4 阶巴特沃斯高通滤波器
def des_bw_filter_hp(cutoff=CUTOFF):  # high pass filter
    b, a = signal.butter(4, Wn=cutoff/NY_FREQ_IDX, btype='highpass')
    return b, a

# 4 阶巴特沃斯带通滤波器
def des_bw_filter_bp(low, high):  # band pass filter
    b, a = signal.butter(4, Wn=(low/NY_FREQ_IDX, high/NY_FREQ_IDX), btype='bandpass')
    return b, a

# 获取趋势特征，通过对信号进行线性回归，得到其斜率lr.coef_[0]
def add_trend_feature(arr, abs_values=False):
    idx = np.array(range(len(arr)))
    if abs_values:
        arr = np.abs(arr)
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]

# sta_lta也就是sta/lta,长短期视窗法，sta短视窗，可以预测短期地震波动，lta长视窗，放映长期趋势
def classic_sta_lta(x, length_sta, length_lta):
    # 计算x^2的累加
    sta = np.cumsum(x ** 2)
    # 转换为浮点型
    sta = np.require(sta, dtype=np.float)
    lta = sta.copy()
    # 通过相减计算sta，和lta,其中length_sta<length_lta
    sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
    sta /= length_sta
    lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
    lta /= length_lta
    
    # sta比lta要短，所以末端补0
    sta[:length_lta - 1] = 0
    # 使用一个较小值代替sta里面的接近0值
    dtiny = np.finfo(0.0).tiny
    idx = lta < dtiny
    lta[idx] = dtiny
    #计算sta/lta
    return sta / lta

In [ ]:
#并行构造特征
def create_features_parallel(seg, mean=1):    
    
    X = pd.DataFrame(index=range(1))
    if mean == 1:
        X["time_to_failure"] = seg.time_to_failure.values[-1]
    
    # 读取信号并且去均值
    xc = pd.Series(seg['acoustic_data'].values)
    xcdm = xc - np.mean(xc)
    
    # 使用截止频率18000做低通滤波
    b, a = des_bw_filter_lp(cutoff=18000)
    xcz = signal.lfilter(b, a, xcdm)
    
    # 做傅里叶变换，并取20000以下的频率做分析
    zc = np.fft.fft(xcz)
    zc = zc[:MAX_FREQ_IDX]

    # FFT变换的实部和虚部
    realFFT = np.real(zc)
    imagFFT = np.imag(zc)
    
    # 每隔2500,一直到20000取频率窗
    freq_bands = [x for x in range(0, MAX_FREQ_IDX, FREQ_STEP)]
    # 通过实部虚部计算fft的幅度
    magFFT = np.sqrt(realFFT ** 2 + imagFFT ** 2)
    # 通过实部虚部计算fft的相位
    phzFFT = np.arctan(imagFFT / realFFT)
    # 相位的负无穷为负二分之π，正无穷为正二分之π
    phzFFT[phzFFT == -np.inf] = -np.pi / 2.0
    phzFFT[phzFFT == np.inf] = np.pi / 2.0
    # 使用0代替nan
    phzFFT = np.nan_to_num(phzFFT)

    # 计算每个频率窗里面的一些统计数据(百分位点数据，幅度的均值，方差，最大值，相位的均值和方差)
    for freq in freq_bands:
        X['FFT_Mag_01q%d' % freq] = np.quantile(magFFT[freq: freq + FREQ_STEP], 0.01)
        X['FFT_Mag_10q%d' % freq] = np.quantile(magFFT[freq: freq + FREQ_STEP], 0.1)
        X['FFT_Mag_90q%d' % freq] = np.quantile(magFFT[freq: freq + FREQ_STEP], 0.9)
        X['FFT_Mag_99q%d' % freq] = np.quantile(magFFT[freq: freq + FREQ_STEP], 0.99)
        X['FFT_Mag_mean%d' % freq] = np.mean(magFFT[freq: freq + FREQ_STEP])
        X['FFT_Mag_std%d' % freq] = np.std(magFFT[freq: freq + FREQ_STEP])
        X['FFT_Mag_max%d' % freq] = np.max(magFFT[freq: freq + FREQ_STEP])

        X['FFT_Phz_mean%d' % freq] = np.mean(phzFFT[freq: freq + FREQ_STEP])
        X['FFT_Phz_std%d' % freq] = np.std(phzFFT[freq: freq + FREQ_STEP])
    
    # 计算实部的统计特性
    X['FFT_Rmean'] = realFFT.mean()
    X['FFT_Rstd'] = realFFT.std()
    X['FFT_Rmax'] = realFFT.max()
    X['FFT_Rmin'] = realFFT.min()
    
    # 计算虚部的统计特性
    X['FFT_Imean'] = imagFFT.mean()
    X['FFT_Istd'] = imagFFT.std()
    X['FFT_Imax'] = imagFFT.max()
    X['FFT_Imin'] = imagFFT.min()
    
    # 计算前6000个实部的统计特性(最多只有20000个)
    X['FFT_Rmean_first_6000'] = realFFT[:6000].mean()
    X['FFT_Rstd__first_6000'] = realFFT[:6000].std()
    X['FFT_Rmax_first_6000'] = realFFT[:6000].max()
    X['FFT_Rmin_first_6000'] = realFFT[:6000].min()
    # 计算前18000个实部的统计特性
    X['FFT_Rmean_first_18000'] = realFFT[:18000].mean()
    X['FFT_Rstd_first_18000'] = realFFT[:18000].std()
    X['FFT_Rmax_first_18000'] = realFFT[:18000].max()
    X['FFT_Rmin_first_18000'] = realFFT[:18000].min()

    # 删除无用的变量清楚缓存
    del xcz
    del zc

    # 之前是对0到18000频率段内所有信号在频率内做分析，下面是每2500hz的频率窗内的时域的信号做分析
    # < 2500
    b, a = des_bw_filter_lp(cutoff=2500)
    xc0 = signal.lfilter(b, a, xcdm)
    # > 2500 < 5000
    b, a = des_bw_filter_bp(low=2500, high=5000)
    xc1 = signal.lfilter(b, a, xcdm)
    # > 5000 < 7500
    b, a = des_bw_filter_bp(low=5000, high=7500)
    xc2 = signal.lfilter(b, a, xcdm)
    # > 7500 < 10000
    b, a = des_bw_filter_bp(low=7500, high=10000)
    xc3 = signal.lfilter(b, a, xcdm)
    # > 10000 < 12500
    b, a = des_bw_filter_bp(low=10000, high=12500)
    xc4 = signal.lfilter(b, a, xcdm)
    # > 12500 < 15000
    b, a = des_bw_filter_bp(low=12500, high=15000)
    xc5 = signal.lfilter(b, a, xcdm)
    # > 15000 < 17500
    b, a = des_bw_filter_bp(low=15000, high=17500)
    xc6 = signal.lfilter(b, a, xcdm)
    # > 17500 < 20000
    b, a = des_bw_filter_bp(low=17500, high=20000)
    xc7 = signal.lfilter(b, a, xcdm)
    # > 20000
    b, a = des_bw_filter_hp(cutoff=20000)
    xc8 = signal.lfilter(b, a, xcdm)
    
    #将9个信号合成一个数组
    sigs = [xc, pd.Series(xc0), pd.Series(xc1), pd.Series(xc2), pd.Series(xc3),
            pd.Series(xc4), pd.Series(xc5), pd.Series(xc6), pd.Series(xc7), pd.Series(xc8)]
    
    # 迭代计算时域统计特性
    for i, sig in enumerate(sigs):
        # 平均值，标准差，最大值，最小值
        X['mean_%d' % i] = sig.mean()
        X['std_%d' % i] = sig.std()
        X['max_%d' % i] = sig.max()
        X['min_%d' % i] = sig.min()
        
        # 离散差值的平均(离散差值，后一个信号减前一个信号所组成的序列)
        X['mean_change_abs_%d' % i] = np.mean(np.diff(sig))
        # 非0的个数
        X['mean_change_rate_%d' % i] = np.mean(np.nonzero((np.diff(sig) / sig[:-1]))[0])
        # 绝对值的最大最小值
        X['abs_max_%d' % i] = np.abs(sig).max()
        X['abs_min_%d' % i] = np.abs(sig).min()
        
        # 前50000个信号的标准差
        X['std_first_50000_%d' % i] = sig[:50000].std()
        # 后50000个信号的标准差
        X['std_last_50000_%d' % i] = sig[-50000:].std()
        # 前1万个信号的标准差
        X['std_first_10000_%d' % i] = sig[:10000].std()
        # 后1万个信号的标准差
        X['std_last_10000_%d' % i] = sig[-10000:].std()
        
        # 平均的特性
        X['avg_first_50000_%d' % i] = sig[:50000].mean()
        X['avg_last_50000_%d' % i] = sig[-50000:].mean()
        X['avg_first_10000_%d' % i] = sig[:10000].mean()
        X['avg_last_10000_%d' % i] = sig[-10000:].mean()
        
        # 最小值的特性
        X['min_first_50000_%d' % i] = sig[:50000].min()
        X['min_last_50000_%d' % i] = sig[-50000:].min()
        X['min_first_10000_%d' % i] = sig[:10000].min()
        X['min_last_10000_%d' % i] = sig[-10000:].min()
        
        # 最大值的特性
        X['max_first_50000_%d' % i] = sig[:50000].max()
        X['max_last_50000_%d' % i] = sig[-50000:].max()
        X['max_first_10000_%d' % i] = sig[:10000].max()
        X['max_last_10000_%d' % i] = sig[-10000:].max()
        
        # 衍生的特征
        X['max_to_min_%d' % i] = sig.max() / np.abs(sig.min())
        X['max_to_min_diff_%d' % i] = sig.max() - np.abs(sig.min())
        # 信号振幅比500大的个数
        X['count_big_%d' % i] = len(sig[np.abs(sig) > 500])
        # 信号求和
        X['sum_%d' % i] = sig.sum()

        X['mean_change_rate_first_50000_%d' % i] = np.mean(np.nonzero((np.diff(sig[:50000]) / sig[:50000][:-1]))[0])
        X['mean_change_rate_last_50000_%d' % i] = np.mean(np.nonzero((np.diff(sig[-50000:]) / sig[-50000:][:-1]))[0])
        X['mean_change_rate_first_10000_%d' % i] = np.mean(np.nonzero((np.diff(sig[:10000]) / sig[:10000][:-1]))[0])
        X['mean_change_rate_last_10000_%d' % i] = np.mean(np.nonzero((np.diff(sig[-10000:]) / sig[-10000:][:-1]))[0])

        X['q95_%d' % i] = np.quantile(sig, 0.95)
        X['q99_%d' % i] = np.quantile(sig, 0.99)
        X['q05_%d' % i] = np.quantile(sig, 0.05)
        X['q01_%d' % i] = np.quantile(sig, 0.01)

        X['abs_q95_%d' % i] = np.quantile(np.abs(sig), 0.95)
        X['abs_q99_%d' % i] = np.quantile(np.abs(sig), 0.99)
        X['abs_q05_%d' % i] = np.quantile(np.abs(sig), 0.05)
        X['abs_q01_%d' % i] = np.quantile(np.abs(sig), 0.01)

        X['trend_%d' % i] = add_trend_feature(sig)
        X['abs_trend_%d' % i] = add_trend_feature(sig, abs_values=True)
        X['abs_mean_%d' % i] = np.abs(sig).mean()
        X['abs_std_%d' % i] = np.abs(sig).std()
        
        # 平均偏离误差
        X['mad_%d' % i] = sig.mad()
        # 峰度(4阶矩)
        X['kurt_%d' % i] = sig.kurtosis()
        # 偏度(3阶矩)
        X['skew_%d' % i] = sig.skew()
        # 中值点
        X['med_%d' % i] = sig.median()
        
        # 希尔伯特变换后的均值
        X['Hilbert_mean_%d' % i] = np.abs(signal.hilbert(sig)).mean()
        # hann窗后的均值
        X['Hann_window_mean'] = (signal.convolve(xc, signal.hann(150), mode='same') / sum(signal.hann(150))).mean()
        
        # 计算sta_lta
        X['classic_sta_lta1_mean_%d' % i] = classic_sta_lta(sig, 500, 10000).mean()
        X['classic_sta_lta2_mean_%d' % i] = classic_sta_lta(sig, 5000, 100000).mean()
        X['classic_sta_lta3_mean_%d' % i] = classic_sta_lta(sig, 3333, 6666).mean()
        X['classic_sta_lta4_mean_%d' % i] = classic_sta_lta(sig, 10000, 25000).mean()
        
        # 计算滑动平均
        X['Moving_average_700_mean_%d' % i] = sig.rolling(window=700).mean().mean(skipna=True)
        X['Moving_average_1500_mean_%d' % i] = sig.rolling(window=1500).mean().mean(skipna=True)
        X['Moving_average_3000_mean_%d' % i] = sig.rolling(window=3000).mean().mean(skipna=True)
        X['Moving_average_6000_mean_%d' % i] = sig.rolling(window=6000).mean().mean(skipna=True)
        
        # 指数加权平均滑动平均
        ewma = pd.Series.ewm
        X['exp_Moving_average_300_mean_%d' % i] = ewma(sig, span=300).mean().mean(skipna=True)
        X['exp_Moving_average_3000_mean_%d' % i] = ewma(sig, span=3000).mean().mean(skipna=True)
        X['exp_Moving_average_30000_mean_%d' % i] = ewma(sig, span=6000).mean().mean(skipna=True)
        
        # 计算一些关于std的特征
        no_of_std = 2
        X['MA_700MA_std_mean_%d' % i] = sig.rolling(window=700).std().mean()
        X['MA_700MA_BB_high_mean_%d' % i] = (
                    X['Moving_average_700_mean_%d' % i] + no_of_std * X['MA_700MA_std_mean_%d' % i]).mean()
        X['MA_700MA_BB_low_mean_%d' % i] = (
                    X['Moving_average_700_mean_%d' % i] - no_of_std * X['MA_700MA_std_mean_%d' % i]).mean()
        X['MA_400MA_std_mean_%d' % i] = sig.rolling(window=400).std().mean()
        X['MA_400MA_BB_high_mean_%d' % i] = (
                    X['Moving_average_700_mean_%d' % i] + no_of_std * X['MA_400MA_std_mean_%d' % i]).mean()
        X['MA_400MA_BB_low_mean_%d' % i] = (
                    X['Moving_average_700_mean_%d' % i] - no_of_std * X['MA_400MA_std_mean_%d' % i]).mean()
        X['MA_1000MA_std_mean_%d' % i] = sig.rolling(window=1000).std().mean()
        
        #
        X['iqr_%d' % i] = np.subtract(*np.percentile(sig, [75, 25]))
        # 得到一个较大值
        X['q999_%d' % i] = np.quantile(sig, 0.999)
        # 得到一个较小值
        X['q001_%d' % i] = np.quantile(sig, 0.001)
        # 裁掉首末0.1的数据后取平均
        X['ave10_%d' % i] = stats.trim_mean(sig, 0.1)
    
    # 对xc取不同滚动窗计算其统计特征
    for windows in [10, 100, 1000]:
        x_roll_std = xc.rolling(windows).std().dropna().values
        x_roll_mean = xc.rolling(windows).mean().dropna().values

        X['ave_roll_std_' + str(windows)] = x_roll_std.mean()
        X['std_roll_std_' + str(windows)] = x_roll_std.std()
        X['max_roll_std_' + str(windows)] = x_roll_std.max()
        X['min_roll_std_' + str(windows)] = x_roll_std.min()
        X['q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
        X['q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
        X['q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
        X['q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
        X['av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
        X['av_change_rate_roll_std_' + str(windows)] = np.mean(
            np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
        X['abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()

        X['ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
        X['std_roll_mean_' + str(windows)] = x_roll_mean.std()
        X['max_roll_mean_' + str(windows)] = x_roll_mean.max()
        X['min_roll_mean_' + str(windows)] = x_roll_mean.min()
        X['q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
        X['q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
        X['q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
        X['q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
        X['av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))
        X['av_change_rate_roll_mean_' + str(windows)] = np.mean(
            np.nonzero((np.diff(x_roll_mean) / x_roll_mean[:-1]))[0])
        X['abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()

    return X

In [ ]:
################################################### TRAIN ###################################################
# 创造特征
rows = 150000
seg = train.iloc[1*rows:1*rows+rows]
res = create_features_parallel(seg, mean=1)
print(res.shape)
res

In [ ]:
################################################### TRAIN ###################################################
segments = int(np.floor(train.shape[0] / rows))
print("Number of segments: ", segments)

# 取15000个样本做训练集
n_samples = 15000

# 放训练集的地方
train_X = pd.DataFrame(index=range(n_samples), columns=create_features_parallel(seg, mean=1).columns, dtype=np.float64)
# train_y = pd.DataFrame(train.time_to_failure.values[::rows][1:], index=range(n_samples), dtype=np.float64, columns=['time_to_failure'])

# 从原始信号中采用15000段信号
seg_gen = (train.iloc[seg_id:seg_id+rows] for seg_id in np.random.randint(train.shape[0]-150000, size=n_samples))
# 对这15000段信号产生数据集
features = Parallel(n_jobs=-1, verbose=2)(delayed(create_features_parallel)(seg, mean=1) for seg in seg_gen) #4.519464272770625
# 显示进度，并把features赋值给train_X
for seg_id, feat in tqdm_notebook(enumerate(features)):
    train_X.loc[seg_id] = feat.values

# 清理内存
del features
gc.collect();

In [ ]:
# 保存数据
cols = [i for i in train_X.columns if i != "time_to_failure"]
train_X[cols].to_csv("train_X_features_865.csv", index=False)
train_y = train_X["time_to_failure"]
train_y.to_csv("train_y.csv", index=False)